In [8]:
%reload_ext autoreload
%autoreload
import pickle, os, json
import torch
from pathlib import Path, PureWindowsPath, PurePosixPath
from util.tools import load_config
import pandas as pd
from sklearn.metrics import classification_report

In [9]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [10]:
config_folder = "config/"
config = load_config(config_folder)
config.__dict__

{'hyperparams': <util.tools.config at 0x24416360490>,
 'programsettings': <util.tools.config at 0x244163603a0>,
 'modelconfig': <util.tools.config at 0x2441635b4c0>}

In [16]:
reports_folder_name = config.programsettings.REPORTS_DIR
data_folder = Path(reports_folder_name)
list_results_files = list(data_folder.glob('**/multi_model_*.pkl'))

all_results = []
all_data_source = []

for results_file in list_results_files:
#     print(results_file)
    with open(results_file,"rb") as f:
        lst_results = pickle.load(f)
#     print("File Name:", results_file, ' \n results: ', lst_results)
    
    for results in lst_results:
        max_seq = 0
        lr = 0
        train_batch = 0
        max_seq = ""
        model_name = ""
        class_weights = '[]'
        kernel_1 = 0
        kernel_2 = 0
        kernel_3 = 0
        if (len(results[0]) > 50):
#             print(results[0])
            results_json = json.loads(results[0])
            max_seq = results_json["hyperparams"]["MAX_SEQ_LENGTH"]
            lr = results_json["hyperparams"]["LEARNING_RATE"]
            train_batch = results_json["hyperparams"]["TRAIN_BATCH_SIZE"]
            model_name = results_json["programsettings"]["MODEL_NAME"]
            layers_freeze = results_json["hyperparams"]["NUM_BERT_LAYERS_FREEZE"]
            try:
                class_weights = results_json["hyperparams"]["LOSS_FN_CLASS_WEIGHTS"]
            
            except: 
                class_weights = '[]'
                
            try:
                kernel_1 = results_json["modelconfig"]["KERNEL_1"]
                kernel_2 = results_json["modelconfig"]["KERNEL_2"]
                kernel_3 = results_json["modelconfig"]["KERNEL_3"]
            
            except: 
                class_weights = '[]'

            try:
                act_function = results_json["modelconfig"]["ACT_FUNCTION"]
                act_fnc_beta = results_json["modelconfig"]["CUST_SFTMX_CLASS_BETA"]
            
            except: 
                act_function = 'softmax'
                act_fnc_beta = 0

        result_store = [results_file, model_name,lr, train_batch,max_seq, layers_freeze, class_weights,act_function,act_fnc_beta, 
                        kernel_1, kernel_2,kernel_3,results[1], 
                        results[2], results[3], results[4], results[5],results[6], results[15], results[16] ]
        all_results.append(result_store)
        
#       Creating list of all dev_labels and dev_preds, so that we can run classification reports for all of them together
        if (len(results) > 7):
            all_data_source.append([results[7],results[8], results[13], results[14], result_store])
                                
    
columns = ['file_name','model_name', 'lr', 'train_batch_size','train_max_seq','layers_freeze','class_weights','act_function', 'act_beta', 
           'Kernel_1','Kernel_2','Kernel_3','train_loss', 
           'dev_loss', 'train_mcc', 'train_f1_score','dev_mcc','dev_f1_score', 'test_mcc', 'test_f1_score' ]

df_results = pd.DataFrame(all_results, columns = columns).sort_values(by='test_f1_score', ascending=False)
pd.set_option('display.max_colwidth', 100)
df_results.to_csv("resultsSummary.csv")
df_results

,file_name,model_name,lr,train_batch_size,train_max_seq,layers_freeze,class_weights,act_function,act_beta,Kernel_1,Kernel_2,Kernel_3,train_loss,dev_loss,train_mcc,train_f1_score,dev_mcc,dev_f1_score,test_mcc,test_f1_score
2,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,384,12,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",softmax,0.000,36,48,60,0.007133,0.040432,0.996642,0.995055,0.985675,0.973941,0.983220,0.967283
1,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,256,12,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",softmax,0.000,36,48,60,0.012626,0.047154,0.993454,0.991550,0.982022,0.968613,0.978794,0.963900
14,reports\re\multi_model_experiment_results_2020-12-03 07_00_34_514754.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 8.0, 1.0]",softmax,0.000,4,3,5,1.387415,1.394221,0.982609,0.971936,0.977693,0.961450,0.975421,0.957198
15,reports\re\multi_model_experiment_results_2020-12-03 23_40_48_770489.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 8.0, 1.0]",softmax,0.000,4,3,5,1.386620,1.392539,0.982708,0.971761,0.978177,0.961535,0.974625,0.955007
24,reports\re\multi_model_experiment_results_2020-12-04 20_08_55_062470.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 8.0, 1.0]",softmax,0.000,24,30,36,1.385970,1.391951,0.983260,0.972379,0.977508,0.959666,0.974291,0.953336
13,reports\re\multi_model_experiment_results_2020-12-02 23_09_28_310531.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 8.0, 1.0]",softmax,0.000,3,4,5,1.387064,1.391204,0.981273,0.967439,0.976381,0.957230,0.973733,0.953336
8,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,128,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",softmax,0.000,36,48,60,0.024254,0.111488,0.985445,0.984262,0.962331,0.950136,0.957104,0.949163
11,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,128,10,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0]",softmax,0.000,36,48,60,0.024373,0.144807,0.985349,0.984061,0.960003,0.947610,0.957237,0.948334
9,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,128,10,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",softmax,0.000,36,48,60,0.024226,0.120995,0.985346,0.984152,0.959243,0.945536,0.957350,0.947572
7,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,24,128,12,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",softmax,0.000,36,48,60,0.025368,0.113962,0.985344,0.983732,0.960120,0.947203,0.957121,0.947241


### Let us generate Classification report for all the things we have available

In [17]:
for i in range(len(all_data_source)):
    dev_labels, dev_preds, test_preds, test_labels, model_config = all_data_source[i]
    print("\n \n \n",model_config)
#     print(classification_report(dev_labels,dev_preds ))
    print(classification_report(test_labels,test_preds ))
    


 
 
 [WindowsPath('reports/re/multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl'), 'BioBERT_fc', 1e-05, 12, 128, 12, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'softmax', 0, 36, 48, 60, 0.024639492770546348, 0.12025048231940862, 0.9851540145895844, 0.9840753730218306, 0.960033703005189, 0.9489194175833915, 0.9560310245333161, 0.946363869595401]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      1838
           1       0.97      0.96      0.96      1813
           2       0.95      0.93      0.94      2154
           3       0.97      0.95      0.96      2045
           4       0.97      0.93      0.95       205
           5       0.96      0.92      0.94      1587
           6       0.98      0.97      0.97      1435
           7       0.88      0.85      0.86       347
           8       0.98      1.00      0.99     13576

    accuracy                           0.97     25000
   macro avg       0.96      0.94      0

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.95      0.93      0.94      3464
           1       0.94      0.98      0.96      3723
           2       0.96      0.98      0.97      4274
           3       0.98      0.98      0.98      4090
           4       0.97      0.84      0.90       420
           5       0.98      0.92      0.95      3024
           6       0.97      0.97      0.97      2802
           7       0.76      0.84      0.80       775
           8       1.00      1.00      1.00     27438

    accuracy                           0.98     50010
   macro avg       0.95      0.94      0.94     50010
weighted avg       0.98      0.98      0.98     50010


 
 
 [WindowsPath('reports/re/multi_model_experiment_results_2020-12-04 16_58_55_892316.pkl'), 'BioBERT_CNN_fc', 1e-05, 12, 256, 8, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 12.0, 1.0], 'softmax', 0.05, 8, 12, 16, 1.394288351097407, 1.394571525549814, 0.9749490681282857, 0.9536961994479368, 0.974171